In [1]:
import numpy as np
import pandas as pd 
import string
import random
import csv
import time
import math
import multiset 

## Reading Data Files 

In [2]:
# The target file includes the original sequences that encodes the data. 
with open('targetBlawat et al.LDPC.txt') as f:
    reads = f.readlines()
for i in range(0,len(reads)):
    reads[i] = reads[i][0:len(reads[i])-1]
# reads 

In [3]:
np.size(reads)

9608

In [4]:
# The errors_shuffled file include the erroneous copies shuffled. 
with open('errors_shuffled.txt') as f:
    reads_err = f.readlines()
for i in range(0,len(reads_err)):
#     reads_err[i] = reads_err[i][0:len(reads_err[i])-1]
    reads_err[i] = reads_err[i].strip()
# reads_err

In [5]:
np.size(reads_err)

1161706

In [6]:
# evyat.txt include the erroneous copies clustered and labeled. 
with open('evyat.txt') as f:
    reads_cl = f.readlines()
for i in range(0,len(reads_cl)):
    reads_cl[i] = reads_cl[i].strip()
# reads_cl

# Functions


In [7]:
def hash_fun_ind(x,a,w,l): 
    cnt = 0 
    for i in range(0,len(x)):
        if x[i] == a[cnt]:
            cnt += 1  
        else:
            cnt = 0 
        if cnt == w:
            return i-w+1 
    return -1

def hash_fun(x,a,w,l):
    ind = hash_fun_ind(x,a,w,l)
    return x[ind:min(len(x),ind+w+l)]

def ind_st(st):
    #All 3-grams: AAA,AAG,AAC,AAT,AGA,AGC,...,TTA,TTG,TTC,TTT
    # A-0, G-1, C-2, T-3
    # index of CAT = Decimal representaion of (203)
    N_q = {"A":0,"C":1,"G":2,"T":3}
    dec = 0
    for i in range(0,len(st)):
        dec += N_q[st[i]]*(4**(len(st)-i-1))
    return dec 
    
def bin_sig(x,q):
    bs = [0]*(4**q) 
    for i in range(0,len(x)-q+1):
        st = x[i:i+q]
        bs[ind_st(st)] = 1
    return bs 
        
def ham_dis(x,y):
    dis = 0
    for i in range(0,len(x)):
        if(x[i]!= y[i]):
            dis += 1
    return dis 

def rand_perm(w):
    return ''.join(random.choice('ACGT') for _ in range(w))

def merge(x,y,parent):
    parent[x]= min(parent[x],parent[y])
    parent[y]= min(parent[x],parent[y])
    return parent
    
def rep_find(inp,parent):
    temp = inp
    cnt = 0
    while(parent[temp]!=temp and cnt<10):
        cnt += 1
        temp=parent[temp]
    return temp 

def edit_dis(s1, s2):
    m=len(s1)+1
    n=len(s2)+1

    tbl = {}
    for i in range(m): tbl[i,0]=i
    for j in range(n): tbl[0,j]=j
    for i in range(1, m):
        for j in range(1, n):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            tbl[i,j] = min(tbl[i, j-1]+1, tbl[i-1, j]+1, tbl[i-1, j-1]+cost)

    return tbl[i,j]

def find_in_arr(el,parent):
    cnt = 0
    for i in range(0,len(parent)):
        if parent[i] == el: 
            cnt += 1 
    return cnt 
        
def readFile(fileName):
        fileObj = open(fileName, "r") #opens the file in read mode
        words = fileObj.read().splitlines() #puts the file into an array
        fileObj.close()
        for i in range(0,len(words)):
            words[i] = int(words[i])
        return words

def read_file_dict(array):
    d = dict(enumerate(array.flatten()))
    return d[0]
    
def display_parent(parent):
    clstr = {}
    for i in range(0,len(parent)):
        clstr[i]=[]
    for i in range(0,len(parent)):
        clstr[rep_find(i,parent)].append(i)
    return clstr

In [8]:
q =  3
x = "AACCTCTAGACACACAACTATTAGTACCTATTATAAACACAACATAAGGTCAAGATAAGCAGAGA"
y = "AACACGAAATAACACACCACGAAGAGACACGACACAACACAACAGAGCACAACAGAGCACAACAC"
ham_dis(bin_sig(x,q),bin_sig(y,q))

23

In [9]:
q

3

In [10]:
edit_dis(x,y)

28

In [11]:
x = reads_err[1]
y = reads_err[215]
ham_dis(bin_sig(x,q),bin_sig(y,q))

7

In [12]:
edit_dis(x,y)

4

# Clustering Algorithm

In [13]:
def rep_in_C(read,C_reps):
    lower = 0;
    upper = len(C_reps) - 1;
    while (lower <= upper):
        mid = lower + int((upper - lower) / 2)
#         print(upper,mid)
        res = -1
        if (read == (C_reps[mid][0])):
            return C_reps[mid][1]
        if (read > (C_reps[mid][0])):
            lower = mid + 1;
        else:
            upper = mid - 1;
    return -1;

def comp_clstrs(alg_clstr,org_clstr,gamma):
    num_exist = 0
    if(len(alg_clstr)>len(org_clstr)):
#         print(alg_clstr)
        return 0 
    else:
        for i in range(0,len(alg_clstr)):
            flg_exist = 0
            for j in range(0,len(org_clstr)):
                if(reads_err[alg_clstr[i]] == org_clstr[j]):
                    flg_exist = 1
                    num_exist +=1
                    break
            if(flg_exist == 0):
                return 0
        if(num_exist < gamma*len(org_clstr)):
            return 0
            
        return 1 
    
def calc_acrcy(parent,C_dict,C_reps,gamma):
    clustering = display_parent(parent)
    acrcy = 0
    for i in range(0,len(clustering)):
        if(len(clustering[i])>=1):
            acrcy = acrcy + comp_clstrs(clustering[i],C_dict[rep_in_C(reads_err[clustering[i][0]],C_reps)],gamma)
    return acrcy

In [14]:
# (Read, Cluster rep of the cluster to which the read belongs to)
C_reps = []
C_dict = {}
rep = reads_cl[0]
for i in range(1,len(reads_cl)):
    if reads_cl[i] != "":
        if reads_cl[i][0] == "*":
            if(len(C_reps)>0):
                C_dict[rep].pop()
                C_reps.pop()
            rep=reads_cl[i-1]
            C_dict[rep]=[]
        else:
#             print(C_dict)
            C_dict[rep].append(reads_cl[i])
            C_reps.append((reads_cl[i],rep))
C_reps.sort(key=lambda x:x[0])

In [52]:
C_reps[0:5]

[('AAAAAAACCGGAATATCAATCGAGCGAATGGGATACTACATTAAGGGATATCACTTGATCACAAG',
  'AACGAAACCGGAATATCAATCGAGCGAATGGGATACTACATTAAGGGATATCACTTGATCACAAG'),
 ('AAAAAAAGGTGAACTCGAAGACGTTAACGATATGACCCGTCAATTAAATGACGTATACACTACT',
  'AACTAAAGGTGAACTCGAAGATCGTTAACGATATGACCCGTCAATTAAATGACGTATACACTACT'),
 ('AAAAACACACTCAACGATACCTCTAACGGATAATCCGGTCACTAACATAAGAAGCACGCAAT',
  'AACTACAAACACACTCAACGATACTCTAACGGATAATCCGGTCACTAACATAAGAAGCACGCAAT'),
 ('AAAAACACCAATCAGATAAGAGTATAAGAGATAAGAACATCTACGAATCCACGCAATATGACG',
  'AACCAAACACCAATCAGATAAGAGTATAAGAGATAAGAACATCTACGAATCCACGCAATATGACG'),
 ('AAAAACACTTAGAAACATCAAGAGCAGCATATCTACCTCGAGTCCGTCTGCACACAACGTAGC',
  'AACTAAACACTTAGAAACATCAAGAGCAGCATATCAACCTCGAGTCCGTCTGCACACAACGTAGC')]

In [53]:
C_dict[list(C_dict.keys())[0]][0:5]

['AATCTCTAGACACACAACTATTAGTACCTATTATAAACACAACATAAGGTCAAGATAAGCAGAGA',
 'AACCTCTAGACACACAACTATTAGTACCTATTATAAACACAACATAAGGTCAAGATAAGCAGAGA',
 'AACCTCTAGACACACAACTATTAGTACCTATTACAAACACAACATAAGGTCAAGATAAGCAGAGA',
 'AACCTCTAGACACACAACTATTAGTACCTATTATAAACACAACATAAGGTCAAGATAAGCAGAGA',
 'AACCTCTAGACAACACAACTATTAGTACCTATTATAAACACAACATAAGGTCAAGATAAGCAGAGA']

In [37]:
C_til  = [0]*(len(reads_err))
parent = [0]*(len(reads_err))
for i in range(0,len(reads_err)):
    C_til[i] = (i,reads_err[i]) 
    parent[i] = i

acrcy_dict = {}
parent_dict = {}
parent_dict[0] = parent.copy() 

gamma = 0.9
local_comm = 100

q = 3
th_high = 12
th_low  = 8
r = 6
w = math.ceil(math.log(len(reads_err[0]),4))
l = math.ceil(math.log(len(reads_err),4))


for lcl_step in range(0,local_comm):   
    a = rand_perm(w)
    hash_C_til = [0]*(len(reads_err))

    for i in range(0,len(C_til)):
        hash_C_til[i] = (i,hash_fun(C_til[i][1],a,w,l))
    
    hash_C_til.sort(key=lambda x:x[1])

    cnt =0
    for i in range(0,len(reads_err)-1):
        if(hash_C_til[i][1] == ""):
            continue 
        else:
            if hash_C_til[i][1] == hash_C_til[i+1][1]:
                if rep_find(i,parent) == rep_find(i+1,parent):
                    continue 
                x = reads_err[hash_C_til[i][0]]
                y = reads_err[hash_C_til[i+1][0]]
                if((ham_dis(bin_sig(x,q),bin_sig(y,q)) <= th_low) or ((ham_dis(bin_sig(x,q),bin_sig(y,q)) <= th_high) and edit_dis(x,y)<=r)):
                    cnt +=  1
                    parent = merge(hash_C_til[i][0],hash_C_til[i+1][0],parent)                    
    acrcy_dict[lcl_step+1]=calc_acrcy(parent,C_dict,C_reps,gamma)
    parent_dict[lcl_step+1]=parent.copy()
    print(acrcy_dict[lcl_step+1]) 
    print(lcl_step,cnt)

107
0 281715
332
1 92751
843
2 201478
1152
3 104436
2010
4 267554
2198
5 71827
2798
6 179121
3585
7 227293
3708
8 33678
3708
9 613
3823
10 29291
4420
11 187749
4781
12 114062
4890
13 33048
5334
14 143210
5657
15 136841
5974
16 134168
6499
17 258154
6603
18 40723
6781
19 130627
7004
20 228290
7004
21 491
7362
22 216801
7549
23 110686
7759
24 158149
8092
25 270857
8098
26 136838
8101
27 130625
8104
28 92748
8224
29 113748
8290
30 74389
8290
31 469
8501
32 233617
8793
33 282081
8945
34 229488
9016
35 189930
9028
36 51220
9061
37 68668
9065
38 136838
9081
39 72274
9109
40 104471
9164
41 109717
9212
42 145933
9265
43 156186
9333
44 239122
9375
45 221419
9380
46 228285
9414
47 119483
9425
48 174422
9445
49 157380
9445
50 618
9463
51 197889
9479
52 107450
9489
53 58548
9508
54 220125
9520
55 162687
9520
56 40723
9521
57 45721
9534
58 175450
9535
59 469
9543
60 179814
9553
61 269288
9553
62 113747
9553
63 81781
9553
64 51092
9558
65 160128
9558
66 844
9559
67 190475
9558
68 116106
9558
69 1161

In [38]:
import json 
with open('parent_dict_12_8_6_120_0.9.json', 'w') as fp:
    json.dump(parent_dict, fp)
with open('acrcy_dict_12_8_6_120_0.9.json', 'w') as fp:
    json.dump(acrcy_dict, fp)

In [40]:
acrcy_dict 

{1: 107,
 2: 332,
 3: 843,
 4: 1152,
 5: 2010,
 6: 2198,
 7: 2798,
 8: 3585,
 9: 3708,
 10: 3708,
 11: 3823,
 12: 4420,
 13: 4781,
 14: 4890,
 15: 5334,
 16: 5657,
 17: 5974,
 18: 6499,
 19: 6603,
 20: 6781,
 21: 7004,
 22: 7004,
 23: 7362,
 24: 7549,
 25: 7759,
 26: 8092,
 27: 8098,
 28: 8101,
 29: 8104,
 30: 8224,
 31: 8290,
 32: 8290,
 33: 8501,
 34: 8793,
 35: 8945,
 36: 9016,
 37: 9028,
 38: 9061,
 39: 9065,
 40: 9081,
 41: 9109,
 42: 9164,
 43: 9212,
 44: 9265,
 45: 9333,
 46: 9375,
 47: 9380,
 48: 9414,
 49: 9425,
 50: 9445,
 51: 9445,
 52: 9463,
 53: 9479,
 54: 9489,
 55: 9508,
 56: 9520,
 57: 9520,
 58: 9521,
 59: 9534,
 60: 9535,
 61: 9543,
 62: 9553,
 63: 9553,
 64: 9553,
 65: 9553,
 66: 9558,
 67: 9558,
 68: 9559,
 69: 9558,
 70: 9558,
 71: 9558,
 72: 9559,
 73: 9564,
 74: 9574,
 75: 9574,
 76: 9574,
 77: 9575,
 78: 9575,
 79: 9575,
 80: 9576,
 81: 9576,
 82: 9577,
 83: 9577,
 84: 9577,
 85: 9576,
 86: 9576,
 87: 9576,
 88: 9576,
 89: 9576,
 90: 9576,
 91: 9576,
 92: 9576,


In [66]:
import plotly.express as px
keys = acrcy_dict.keys()
values = acrcy_dict.values()

px.scatter(x=keys,y=values)

In [41]:
# Returns the number of times an element occurs in a list
find_in_arr(994,parent_dict[99])

0

In [54]:
clustering1 = display_parent(parent_dict[77])
clustering1[0]

[0,
 9872,
 21601,
 23924,
 25815,
 48380,
 50272,
 51978,
 113706,
 159978,
 174077,
 176345,
 197538,
 209200,
 209577,
 214115,
 225798,
 226294,
 227588,
 234524,
 246956,
 258959,
 262767,
 262981,
 273525,
 277238,
 281451,
 289510,
 289529,
 312364,
 315654,
 358120,
 359534,
 383872,
 386324,
 397438,
 422114,
 424832,
 469150,
 470816,
 474113,
 476475,
 480454,
 486103,
 501504,
 508355,
 520956,
 532232,
 536392,
 538353,
 538926,
 543350,
 546103,
 559236,
 571839,
 632881,
 640682,
 649480,
 668177,
 680060,
 681985,
 727081,
 745442,
 754649,
 770806,
 776832,
 780117,
 796973,
 811466,
 822770,
 831203,
 855636,
 855882,
 881473,
 888676,
 892619,
 914870,
 926108,
 926739,
 932556,
 934436,
 935009,
 940439,
 951968,
 958551,
 975463,
 996552,
 1000017,
 1006128,
 1035903,
 1058670,
 1114543,
 1115259,
 1136864,
 1140122,
 1146995]

In [55]:
clustering2 = display_parent(parent_dict[78])
clustering2[0]

[0,
 9872,
 21601,
 23924,
 25815,
 48380,
 50272,
 51978,
 113706,
 159978,
 174077,
 176345,
 197538,
 209200,
 209577,
 214115,
 225798,
 226294,
 227588,
 234524,
 246956,
 258959,
 262767,
 262981,
 273525,
 277238,
 281451,
 289510,
 289529,
 312364,
 315654,
 358120,
 359534,
 383872,
 386324,
 397438,
 422114,
 424832,
 469150,
 470816,
 474113,
 476475,
 480454,
 486103,
 501504,
 508355,
 520956,
 532232,
 536392,
 538353,
 538926,
 543350,
 546103,
 559236,
 571839,
 632881,
 640682,
 649480,
 668177,
 680060,
 681985,
 727081,
 745442,
 754649,
 770806,
 776832,
 780117,
 796973,
 811466,
 822770,
 831203,
 855636,
 855882,
 881473,
 888676,
 892619,
 914870,
 926108,
 926739,
 932556,
 934436,
 935009,
 940439,
 951968,
 958551,
 975463,
 996552,
 1000017,
 1006128,
 1035903,
 1058670,
 1114543,
 1115259,
 1136864,
 1140122,
 1146995]

In [56]:
clustering1 == clustering2

False

In [57]:
parent_dict[0]==parent_dict[3] 

False

In [35]:
# !git status 

In [36]:
# !git add Clustering-algorithm-single-core-\(120,0.9\).ipynb

In [34]:
# !git add -u && git status && git commit -m "fixed bugs in acrcy_dict" && git push origin master 

On branch master
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	deleted:    Clustering -Copy1.ipynb
	deleted:    Clustering -Copy2.ipynb
	new file:   Clustering-algorithm-single-core-(120,0.9).ipynb
	modified:   Clustering-parent_array.ipynb
	deleted:    parent_file_12_6_6.txt
	deleted:    parent_file_16_8_6.txt

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/
	Clustering-algorithm-accuracy-measure.ipynb
	Clustering-algorithm-single-core-(200,0.6).ipynb
	acrcy_dict_12_8_6_120_0.9.json
	errors_shuffled.txt
	evyat.txt
	nips-2017-camera-long.pdf
	parent_dict_12_8_6_120_0.9.json
	parent_file_12_8_6.txt
	parent_file_12_8_6_120_0.9.txt
	parent_file_12_8_6_acrcy.txt
	parent_file_12_8_6_acrcy_120_0.9.txt
	parent_file_12_8_6_ver2.txt
	targetBlawat et al.LDPC.txt
	../Scaling of Errors with cluster size/

[master 95e8b74] fixed bugs in acrcy_dict
 6 files changed, 283364 insertions(+), 2569276 deletions(-)
 dele